# Importing Libraries and Cleaned Datasets

In [67]:
#import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import statsmodels as sm

gini = pd.read_csv('gini_index.csv')
ltss_2022 = pd.read_csv('ltss_combined.csv')
socio_economic = pd.read_csv('socio_econ.csv')
fips = pd.read_csv('all_fips.csv')

## Quick Prelimninary cleaning and prepping for analysis 

In [68]:
gini = gini.drop(columns=['state_fips', 'county_fips'])
gini['fips'] = gini['fips'].astype(str)
socio_economic['fips'] = socio_economic['fips'].astype(str)

In [69]:
# set indices
socio_economic.set_index(['state', 'fips'])
ltss_2022.set_index('state')
gini.reset_index(inplace=True)

### Troubleshoot merge issues with gini index by merging with 'all_fips'

In [70]:
gini.set_index(['state', 'county'])
fips.set_index(['state', 'county'])

fips state_abbr
state  county                                      
Alaska Aleutians East Borough       2013         AK
       Aleutians West Census Area   2016         AK
       Anchorage Municipality       2020         AK
       Bethel Census Area           2050         AK
       Bristol Bay Borough          2060         AK
...                                  ...        ...
NaN    VIRGINIA                    51000        NaN
       WASHINGTON                  53000        NaN
       WEST VIRGINIA               54000        NaN
       WISCONSIN                   55000        NaN
       WYOMING                     56000        NaN

[3195 rows x 2 columns]

In [71]:
gini_test = pd.merge(gini, fips, left_index=True, right_index=True)
gini_index = gini_test.drop(columns=['index', 'fips_y', 'county_y', 'state_y'])

### Normalize updated 'gini_index' to merge with 'socio_economic'

In [72]:
gini_index.reset_index(inplace=True)
gini_index = gini_index.rename(columns={'fips_x': 'fips', 'county_x': 'county', 'state_x': 'state'})

## Merge for analysis

In [73]:
ltss_socioecon = pd.merge(socio_economic, ltss_2022, how='left', on='state')

In [74]:
combined_socioecon = pd.merge(ltss_socioecon, gini_index,  left_index=True, right_index=True)

In [ ]:
combined_socioecon

## Clean new table

In [77]:
# Drop columns: 'county_y', 'state_y' and 2 other columns
combined_socioecon = combined_socioecon.drop(columns=['county_y', 'state_y', 'fips_y', 'state_abbr_y'])

In [78]:
combined_socioecon = combined_socioecon.rename(columns={'county_x': 'county', 'state_x': 'state', 'fips_x': 'fips', 'state_abbr_x': 'state_abbr'})

In [80]:
combined_socioecon.to_csv('combined_socioecon.csv', index=False)